# Gradient Boosting: Appliance and Optimization

The field of Machine Learning (ML) implements quite a lot of different algorithms to achieve domain specific goals. Bioinformatics is one of those domain specifics where we try to create a **descision space** based on biomarkers (a set of features that result from biological results, allowing us to classify an organism as *A* or *B*).

Within the field of bioinformatics, we often work with both **Structured** and **Unstructured** data. These are the 2 main classification by which ML engineers decide on an algorithm. Structured data is acquired from locations such as: **SQL**, **CSV (Excel)** and **Text** documents. **Unstructured** data is commonly referred to as Images or Speech, which is very hard to model with Structured Oriented algorithms.

For our specific project, we work with **Structured** data (Yay!). Structured data allows for *a lot* of different implementations, but the most used implementation is a **Desicion Tree Classifier (DTC)**. A DTC utilizes the idea of *Gini Impurity* (most generally): trying to find a path through a given **if ... else ...** that provides the most robust classification result.

However, a major drawback of DTCs is **overfitting**. Overfitting means: The model takes too much **Variance** of data into consideration when deciding on its **decision space**. Yet, we would like to build a model that *generalizes* over our training data so we can apply it as broadly to our problem as possible.

This major DTC problem is prevented by using **Gradient Boosting**. Gradient Boosting tries to find the optimal DTC by *iteratively* creating new, and most importantly, better DTCs. The allocated [Gradient Descent](https://en.wikipedia.org/wiki/Gradient_descent) algorithm ensure we move the tree into a 'negative' manner (gradient **descent**), allowing us to find the DTC with the least amount of prediction loss.

But: iteratively getting the best DTC? Doesn't this mean it will **always** overfit to the data?! Well, no. Gradient Boosting implements an **Overfitting Detection** technique through *Cross-Validation* (Let me know if you need futher explanation on CV, but I will skip it for now). As soon as the optimized DTC starts to get an accuracy estimation (e.g. AUC) from its respective CV, and the difference between the previous scoring and the current scoring is worse than a given threshold *Y*, the iterations will hault and the respective DTC is returned.

*Why this specific metric to detect overfitting?* If you remember the overfitting definition (taking too much variance into account): the validation set will perform a lot worse for an overfitted model than a generalized model. It's overfitting so drammatically, it can't even predict a dataset that looks a lot like our training data!

Combining all of these methods makes Gradient Boosting so incredibly powerful. You simply preprocess the data, create a model object and let the framework do the rest :)

## Imports

To implement all of the gibberish above, we need to use some essential Python Libraries.

- **typing** is used to declare *types* in python. Ensuring we do not randomly combine data objects.

- **NumPy** is an optimized matrix calculator.
- **Pandas** is an extremely common Data Wrangler library that allows us to easily interact with the Structured data
- **SKlearn** is a common Machine Learning Library that allows us to quickly produce Pipelines
- **CatBoost** is a relatively new competitor in the field of Gradient Boosting but very powerfull by utilizing the lastest and greatest research findings.
- **XGBoost** is the most widely used Gradient Boosting framework in the field of ML. A 'no-brainer' for our comparisons!

- **custom_functions** simply provides some helper-functions :)

In [1]:
from typing import *

import numpy as np
import pandas as pd
import sklearn as sk
import catboost as cb
import xgboost as xgb

from custom_functions import (
    store_json,
    load_json,
)

In [2]:
# Additionally, I will set some constants. 
# This is a pythonic way of writing code since 
# it makes it easier for new developers to 
# quickly change recurring values.

DROPPABLE_COLUMNS:List = ["Unnamed: 0",]
TEST_SIZE:float = 0.20
RANDOM:int = 44

# Normally, I would add the model params
# as well but for the sake of structure:
# I put them with their respective models

## Data Acquisition

Normally, preprocessing is an **essential** step to allow for (effective) predictive modelling. Yet, this has already been done for us! (Class imbalance, independent variable distribution, etc etc). So, all we need to do is getting our Structured Data ready for the model.

Machine Learning works with **Instances** and **Features**. e.g. in a research on biomarkers for Human cancer classification, humans are the instances and the different genes are features. Where a gene count presents the feature value (in our case, -1/0/1/2). Our project requirement is the **classification of each array into a specific tumor subtype**. Therefore, the Arrays become the instances and the different chromosomal regions become the features.

The whole Instance vs Features stuff has been performed within the 'PreAnalysis' notebook. So we only need to access the processed data using Pandas.

In [3]:
df = pd.read_csv("./data/superset.csv")
df = (df
    .set_axis([df["Unnamed: 0"]]) # A common artefact of pandas since it tries to reset the index
    .drop(columns=DROPPABLE_COLUMNS) # Now we can delete the artefact since it's redundant (values are now inside of the index)
)

In [4]:
df.Subgroup = df.Subgroup.replace(
    {cat:num for num, cat in enumerate(df.Subgroup.unique())}
    )
# An ML model doesn't really understand strings (text) like we do. 
# Therefore, we simply transform the individual string values in a simple numerical value (1 = HER,2 = HR, etc etc). 

In [5]:
df.shape # 100 Instances with 2835 features each

(100, 2835)

In [6]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,2825,2826,2827,2828,2829,2830,2831,2832,2833,Subgroup
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,...,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.00,100.000000,100.000000,100.00000
mean,0.030000,-0.050000,-0.030000,0.010000,-0.030000,-0.040000,-0.040000,-0.100000,-0.10000,-0.100000,...,1.130000,1.040000,1.130000,1.09000,1.120000,1.130000,1.15,1.130000,1.120000,1.00000
std,0.481055,0.457817,0.459578,0.481894,0.521362,0.490722,0.510891,0.541229,0.57735,0.522233,...,0.463953,0.530294,0.463953,0.51434,0.455716,0.485237,0.50,0.525222,0.498077,0.80403
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.00,1.000000,1.000000,0.00000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.00,1.000000,1.000000,1.00000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.00,1.000000,1.000000,2.00000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.00000,1.000000,...,2.000000,2.000000,2.000000,2.00000,2.000000,2.000000,2.00,2.000000,2.000000,2.00000


The describe function shows a nice recurring pattern of mean = 0.03 or -0.05 or -0.01 etc etc. AKA there is nothing weird going on, which is very nice :3

## Data Splitting

Since we would like to test the accuracy of our model, it's vital to have a **test set**. This allows us to score the model's accuracy on a similar, but not identical, dataset. SKlearn has an integrated *train_test_split* function that will take of this process. We only need to define the test set fraction and the random state. The fraction is currently at 0.25 (creating 25 test samples) with a random state of 44.

Normally, such little test size would cause severe problems for a production model. But we are limited to 100 instances so it's all we are gonna get.

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
# Using an underscore so no1 accidently uses the original dataset
X_ = df.drop(columns=["Subgroup"])
y_ = df.Subgroup

# Initial Datasets
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=TEST_SIZE, random_state=RANDOM)

# p.s. The reason for random_state has to do with reproducibility. SKlearn utilizes random permutations that bases its number on the provided seed. 
# This is not essential for our project but is a very good practice to remember for a scientific setting.

Not all frameworks apply Cross-Validation within their training pipeline. Therefore, we manually split some data from the training set to act as *validation data*. The model can use this to score its intermediate evaluations without being exposed to the final test set.

In [9]:
X_trainset, X_val, y_trainset, y_val = train_test_split(X_train, y_train, test_size=TEST_SIZE, random_state=RANDOM)

# Building The Different Models

Now comes the fun part! Let's build some models :D

We will apply the following frameworks for comparison (besides our 'Simple Baseline Model'):
- Sklearn
- CatBoost
- XGBoost

## Sklearn

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
SKGBC_params = {
    'learning_rate':[0.1, 0.05],
    'n_estimators': [10, 100, 1000], 
    'max_leaf_nodes': [2, 4, 6, 8, 10, 12], 
    'max_depth': [2, 4, 6, 8], 
    'min_samples_split': [2, 5, 8, 11]
}

# If you wish to add more items
# params.update({
#     'one':[],
#     'two':[],
#     'three'"(),
# })

In [12]:
SKGBC = GradientBoostingClassifier(random_state=RANDOM)

optimized_SKGBC = GridSearchCV(SKGBC, SKGBC_params)

optimized_SKGBC.fit(X_trainset, y_trainset)

KeyboardInterrupt: 

In [ ]:
# A Python f-string. Used to implement data object content inside of text.
# The triple " makes it a multistring, remembering the layout.
f"""
Sklearn GridSearch resulted with {optimized_SKGBC.best_score_}, according to the {optimized_SKGBC.scorer_} scoring metric.

The following parameters were applied:

{optimized_SKGBC.best_params_}
"""

In [ ]:
optimized_SKGBC.best_estimator_ # Is the actual best model

## CatBoost, Optimized and Baseline

https://catboost.ai/docs/concepts/about.html

In [ ]:
print(cb.__version__) # Consistency

0.25.1


In [ ]:
# Apply a validation-like training set
train_set = cb.Pool(X_trainset, label=y_trainset)
eval_set = cb.Pool(X_val, label=y_val)

entire_train = cb.Pool(X_train, label=y_train)

In [ ]:
# # Set grid search params
# cbc_params = {
#     'random_seed': 88,
#     'verbose': 100,
#     'loss_function': 'MultiClass'}

# CBC = cb.CatBoostClassifier(**cbc_params)

# grid_params ={
#     'iterations': [100,500,1000],
#     'learning_rate': [0.01,0.05,0.1],
#     'l2_leaf_reg': [1,3,5,7,9,11,20],
#     'depth': [6, 8, 10],
#     'random_strength': [0.01, 0.05, 0.1]}

# # Grid Search does automatic CV. So we only need to parse the initial training data
# CBC_grid = CBC.grid_search(grid_params, X=entire_train, partition_random_seed=12)

In [ ]:
CBC_grid['params']

{'depth': 6,
 'l2_leaf_reg': 7,
 'iterations': 1000,
 'random_strength': 0.05,
 'learning_rate': 0.05}

In [ ]:
store_json(CBC_grid, "./data/hyper_catboost.json")

Stored file at ./data/hyper_catboost.json


TODO

sklearn.GBC vs catboost.GBC vs xgb.GBC

plot loss function metric for all classifiers in one plot

confusion matrix

In [ ]:
f"{round(100 - len(*np.nonzero(y_preds - y_val.values)) / len(y_preds) * 100)}% Accuracy"

'79% Accuracy'

### Default CatBoost Test

### Default CatBoost Decomp

In [ ]:
avg_expected_loss_CBC, avg_bias_CBC, avg_var_CBC = bias_variance_decomp(
        default_CBC, X_trainset.values, y_trainset.values, X_val, y_val, # Used .values since mlxtend extracts better from numpy arrays than pandas dfs
        loss='0-1_loss',
        random_seed=66)

print(f'Average expected loss: {avg_expected_loss_CBC}')
print(f'Average bias: {avg_bias_CBC}')
print(f'Average variance: {avg_var_CBC}')

1:	learn: 1.0426174	total: 151ms	remaining: 1m 15s
2:	learn: 1.0160193	total: 230ms	remaining: 1m 16s
3:	learn: 1.0011996	total: 294ms	remaining: 1m 13s
4:	learn: 0.9774458	total: 381ms	remaining: 1m 15s
5:	learn: 0.9461042	total: 456ms	remaining: 1m 15s
6:	learn: 0.9244746	total: 529ms	remaining: 1m 15s
7:	learn: 0.9097368	total: 600ms	remaining: 1m 14s
8:	learn: 0.8876313	total: 676ms	remaining: 1m 14s
9:	learn: 0.8647059	total: 750ms	remaining: 1m 14s
10:	learn: 0.8488065	total: 829ms	remaining: 1m 14s
11:	learn: 0.8320039	total: 899ms	remaining: 1m 13s
12:	learn: 0.8142801	total: 981ms	remaining: 1m 14s
13:	learn: 0.7965193	total: 1.06s	remaining: 1m 14s
14:	learn: 0.7805144	total: 1.13s	remaining: 1m 14s
15:	learn: 0.7692651	total: 1.22s	remaining: 1m 14s
16:	learn: 0.7545986	total: 1.28s	remaining: 1m 13s
17:	learn: 0.7379340	total: 1.35s	remaining: 1m 13s
18:	learn: 0.7162706	total: 1.43s	remaining: 1m 14s
19:	learn: 0.7063683	total: 1.51s	remaining: 1m 14s
20:	learn: 0.6923661	

ValueError: could not broadcast input array from shape (19,1) into shape (19)

### SKlearn Based GBC

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
hard_params = {'n_estimators': 1000, 'max_leaf_nodes': 4, 'max_depth': None, 'random_state': 2,
                'min_samples_split': 5}

# intermediate params

params = dict(hard_params)
# params.update(inter_params)

In [ ]:
# X_trainset, X_val, y_trainset, y_val
GBC = GradientBoostingClassifier(**params)
GBC.fit(X_trainset, y_trainset)

GradientBoostingClassifier(max_depth=None, max_leaf_nodes=4,
                           min_samples_split=5, n_estimators=1000,
                           random_state=2)

In [ ]:
GBC.score(X_val, y_val)

0.8421052631578947

## attempt to plot the training vs validation to identify overfitting


In [ ]:
# WIP
# attempt to plot the training process(https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html)
# training vs validation deviance


y_val_list = y_val.to_numpy()



test_score = np.zeros((params['n_estimators'],), dtype=np.float64)
for i, y_pred in enumerate(GBC.staged_predict(X_val)):
    test_score[i] = GBC.loss_(y_val_list, y_pred)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, reg.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')
fig.tight_layout()
plt.show()


ValueError: operands could not be broadcast together with shapes (19,3) (19,) 

### SKlearn decomposition

In [ ]:
from mlxtend.evaluate import bias_variance_decomp

In [ ]:
def_GBC = GradientBoostingClassifier(**params)

avg_expected_loss_GBC, avg_bias_GBC, avg_var_GBC = bias_variance_decomp(
        def_GBC, X_trainset.values, y_trainset.values, X_val, y_val, # Used .values since mlxtend extracts better from numpy arrays than pandas dfs
        loss='0-1_loss',
        random_seed=56)

print(f'Average expected loss: {avg_expected_loss_GBC}')
print(f'Average bias: {avg_bias_GBC}')
print(f'Average variance: {avg_var_GBC}')

Average expected loss: 0.22289473684210523
Average bias: 0.15789473684210525
Average variance: 0.16210526315789475


## k-fold cross-validation:
It seems like you can implement the cross-validation for the train-validation split in this way. (Use to evaluate different models using k-fold CV on the training set
 & pick the final model, which is then used to predict the test set!)

 *Question*: 

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

model = GBC
kfold = KFold(n_splits=7,shuffle = True, random_state=10)
results = cross_val_score(model, X_trainset, y_trainset, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 76.79% (12.37%)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c915e4f9-60c2-40b5-a522-8a90cb3fd50a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>